<a href="https://colab.research.google.com/github/srinivasrdhkrshnn/CS6910-Assignment-2/blob/main/Assignment_2_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
src_url = "https://storage.googleapis.com/wandb_datasets/nature_12K.zip"
src_zip = "nature_12K.zip"
DATA_TRAIN_SRC = "inaturalist_12K/train" 
DATA_TEST_SRC = "inaturalist_12K/val"
TRAIN_IMAGES_PER_LABEL = 1000
TEST_IMAGES_PER_LABEL = 200
BALANCED_SPLITS = {"train" : 900, "val" : 100}

In [ ]:
%%capture
!curl -SL $src_url > $src_zip
!unzip $src_zip

**Import Libraries & Install, Import WandB Library and Login to WandB account**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import matplotlib.pyplot as plt
from random import shuffle

!pip3 install tensorflow -qqq
!pip3 install wandb -qqq
import wandb
!wandb login
from wandb.keras import WandbCallback

In [ ]:
PROJECT_NAME = "CS6910 ASSIGNMENT 2B"

**Upload Raw Train Data**

In [ ]:
# source directory for all raw train data
SRC_TRAIN = DATA_TRAIN_SRC
# number of images per class label
# the total number of images is 10X this (10 classes)
TOTAL_IMAGES = TRAIN_IMAGES_PER_LABEL * 10
PREFIX_1 = "train" # convenient for tracking local data

In [ ]:
TRAIN_RAW_DATA_AT = "_".join([PREFIX_1, "raw_data", str(TOTAL_IMAGES)])
run = wandb.init(project=PROJECT_NAME, job_type="upload")

# create an artifact for all the raw data
raw_data_at = wandb.Artifact(TRAIN_RAW_DATA_AT, type="raw_data")

# SRC_DIR contains 10 folders, one for each of 10 class labels
# each folder contains images of the corresponding class
labels = os.listdir(SRC_TRAIN)
for l in labels:
  imgs_per_label = os.path.join(SRC_TRAIN, l)
  if os.path.isdir(imgs_per_label):
    imgs = os.listdir(imgs_per_label)
    # randomize the order
    shuffle(imgs)
    img_file_ids = imgs[:TRAIN_IMAGES_PER_LABEL]
    for f in img_file_ids:
      file_path = os.path.join(SRC_TRAIN, l, f)
      # add file to artifact by full path
      raw_data_at.add_file(file_path, name=l + "/" + f)

# save artifact to W&B
run.log_artifact(raw_data_at)
run.finish()

**Upload Raw Test Data**

In [ ]:
# source directory for all raw train data
SRC_TEST = DATA_TEST_SRC
# number of images per class label
# the total number of images is 10X this (10 classes)
TOTAL_IMAGES = TEST_IMAGES_PER_LABEL * 10
PREFIX_2 = "test" # convenient for tracking local data

In [ ]:
TEST_RAW_DATA_AT = "_".join([PREFIX_2, "raw_data", str(TOTAL_IMAGES)])
run = wandb.init(project=PROJECT_NAME, job_type="upload")

# create an artifact for all the raw data
raw_data_at = wandb.Artifact(TEST_RAW_DATA_AT, type="raw_data")

# SRC_DIR contains 10 folders, one for each of 10 class labels
# each folder contains images of the corresponding class
labels = os.listdir(SRC_TEST)
for l in labels:
  imgs_per_label = os.path.join(SRC_TEST, l)
  if os.path.isdir(imgs_per_label):
    imgs = os.listdir(imgs_per_label)
    # randomize the order
    shuffle(imgs)
    img_file_ids = imgs[:TEST_IMAGES_PER_LABEL]
    for f in img_file_ids:
      file_path = os.path.join(SRC_TEST, l, f)
      # add file to artifact by full path
      raw_data_at.add_file(file_path, name=l + "/" + f)

# save artifact to W&B
run.log_artifact(raw_data_at)
run.finish()

**Split Train Data into Train and Validation**

In [ ]:
run = wandb.init(project=PROJECT_NAME, job_type="data_split")

# find the most recent ("latest") version of the full raw data
# you can of course pass around programmatic aliases and not string literals
data_at = run.use_artifact(TRAIN_RAW_DATA_AT + ":latest")
# download it locally (for illustration purposes/across hardware; you can
# also sync/version artifacts by reference)
data_dir = data_at.download()

# create balanced train, val, test splits
# each count is the number of images per label
DATA_SPLITS = BALANCED_SPLITS

ats = {}
# wrap artifacts in dictionary for convenience
for split, count in DATA_SPLITS.items():
  ats[split] = wandb.Artifact("_".join([PREFIX_1, split, "data", str(count*10)]), 
                              "_".join([split, "data"]))

labels = os.listdir(data_dir)
for l in labels:
  if l.startswith("."): # skip non-label file
    continue
  imgs_per_label = os.listdir(os.path.join(data_dir, l))
  shuffle(imgs_per_label)
  start_id = 0
  for split, count in DATA_SPLITS.items():
    # take a subset
    split_imgs = imgs_per_label[start_id:start_id+count]
    for img_file in split_imgs:
      full_path = os.path.join(data_dir, l, img_file)
      # add file to artifact by full path
      # note: pass the label to the name parameter to retain it in
      # the data structure 
      ats[split].add_file(full_path, name = os.path.join(l, img_file))
    start_id += count

# save all three artifacts to W&B
# note: yes, in this example, we are cheating and have labels for the "val" data ;)
for split, artifact in ats.items():
  run.log_artifact(artifact)

run.finish()

**Default Configuration**

In [ ]:

n_train = BALANCED_SPLITS["train"] * 10
n_val = BALANCED_SPLITS["val"] * 10
n_epochs = 7
img_size = 229
batchsize = 180

**Define and Compile Model**

In [ ]:
def cnn_model(fc_size,pretrained_model):
   

   if pretrained_model=='InceptionV3':
     base = tf.keras.applications.InceptionV3(include_top=False,input_shape=(img_size,img_size,3),weights='imagenet')
   elif pretrained_model=='Xception':
     base = tf.keras.applications.Xception(include_top=False,input_shape=(img_size,img_size,3),weights='imagenet')
   elif pretrained_model=='ResNet50':
     base = tf.keras.applications.ResNet50(include_top=False,input_shape=(img_size,img_size,3),weights='imagenet')
   elif pretrained_model=='InceptionResNetV2':
     base = tf.keras.applications.InceptionResNetV2(include_top=False,input_shape=(img_size,img_size,3),weights='imagenet')
   elif pretrained_model=='MobileNetV2':
     base = tf.keras.applications.MobileNetV2(include_top=False,input_shape=(img_size,img_size,3),weights='imagenet')      
   else:
     raise Exception('no pretrained model given')
   
   for layer in base.layers:
     layer.trainable = False

   x = base.output
   x = tf.keras.layers.GlobalAveragePooling2D()(x)
   x = tf.keras.layers.Dense(fc_size, activation='relu')(x)
   prediction = tf.keras.layers.Dense(10, activation='softmax')(x)

   model = Model(inputs=base.input, outputs=prediction)
  
   model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
   model.summary()
   return model


**Train Model**

In [ ]:
def train():
  
  config_defaults = {
      "model_name" : "InceptionV3",
      "fc_size" : 1024,
      "augmentation" : 1,
  }

  # track this experiment with wandb: all runs will be sent to the given project name
  run = wandb.init(config=config_defaults,job_type='train')
  cfg = wandb.config
  

  MODEL_NAME = cfg.model_name 
  INIT_MODEL_DIR = "init_model_" + cfg.model_name
  FINAL_MODEL_DIR = "final_model_" + cfg.model_name

  # artifact names
  train_at = os.path.join(PROJECT_NAME, PREFIX_1 + "_train_data_" + str(n_train)) + ":latest"
  val_at = os.path.join(PROJECT_NAME, PREFIX_1 + "_val_data_" + str(n_val)) + ":latest"

  train_data = run.use_artifact(train_at, type='train_data')
  train_dir = train_data.download()
  val_data = run.use_artifact(val_at, type='val_data')
  val_dir = val_data.download()

  # create augmented train and validation data generators
  if cfg.augmentation == 1:
    train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=36,
      shear_range=0.3,
      zoom_range=0.4,
      horizontal_flip=True,
      fill_mode='nearest')
  else  :
    train_datagen = ImageDataGenerator(rescale = 1. /255)
  
  val_datagen = ImageDataGenerator(rescale = 1. / 255)

  train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size,img_size),
    batch_size = batchsize,
    class_mode='categorical')

  val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_size,img_size),
    batch_size = batchsize,
    class_mode='categorical')
  

  # instantiate model and callbacks
  model = cnn_model(cfg.fc_size,cfg.model_name)

  model_artifact = wandb.Artifact(
            cfg.model_name, type="model",
            description="unmodified model",
            metadata=dict(cfg))

  model.save(INIT_MODEL_DIR)
  model_artifact.add_dir(INIT_MODEL_DIR)
  run.log_artifact(model_artifact)
  callbacks = [WandbCallback()]
 
  # train and validate
  model.fit(
      train_generator,
      steps_per_epoch = n_train // batchsize,
      epochs = n_epochs,
      validation_data=val_generator,
      callbacks = [WandbCallback()],
      validation_steps = n_val // batchsize
      )

  # save trained model as artifact
  trained_model_artifact = wandb.Artifact(
            MODEL_NAME, type="model",
            description="finetuned model",
            metadata=dict(cfg))

  model.save(FINAL_MODEL_DIR)
  trained_model_artifact.add_dir(FINAL_MODEL_DIR)
  run.log_artifact(trained_model_artifact)
  run.finish()

In [ ]:
def sweeper(sweep_config,PROJECT_NAME):
  sweep_id=wandb.sweep(sweep_config,project=PROJECT_NAME)
  wandb.agent(sweep_id,train,project=PROJECT_NAME)

In [ ]:
#sweep dictionary
sweep_config={
    'method':'bayes',
    'metric':{
        'name':'val_accuracy',
        'goal':'maximize'},

}

parameters_dict={
    
    'fc_size':{
        'values':[1024,512,256]
    },
    
    'augmentation':{
      'values':[1]  
    },
    
    'model_name':{
        'values':['ResNet50','InceptionResNetV2','InceptionV3','Xception','MobileNetV2']
    },
}

sweep_config['parameters']=parameters_dict

In [ ]:
sweeper(sweep_config,PROJECT_NAME)